In [ ]:
import pandas as pd
import torch
from transformers import MobileNetV2ForImageClassification, AutoModelForImageClassification, MobileNetV2Config

In [3]:
unique_labels = [0, 1, 2]
class_names = ['angular_leaf_spot', 'bean_rust', 'healthy']

def calculate_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def measure_inference_speed(model, input_data):
    with torch.no_grad():
        start_time = torch.cuda.Event(enable_timing=True)
        end_time = torch.cuda.Event(enable_timing=True)

        start_time.record()
        _ = model(input_data)
        end_time.record()

        torch.cuda.synchronize()
        return start_time.elapsed_time(end_time)

id2label = {i: name for i, name in zip(unique_labels, class_names)}
label2id = {v:k for k, v in id2label.items()}
num_classes = len(label2id)
num_labels = 3
# mobile_net_v2_config = MobileNetV2Config(num_labels=num_labels)
# mobile_net_v2_model = MobileNetV2ForImageClassification(mobile_net_v2_config)
# mobile_net_v2_model.to('cuda')

model_name = "facebook/deit-tiny-patch16-224" 
mobile_net_v2_model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
mobile_net_v2_model.to('cuda')
vit_model = AutoModelForImageClassification.from_pretrained(
    'merve/beans-vit-224',
    num_labels=num_labels,
    ignore_mismatched_sizes=True
).to('cuda')

models = [
    ("DEiT", mobile_net_v2_model),
    ("ViT", vit_model), 
]

data = []

for model_name, model in models:
    num_params = calculate_parameters(model)

    input_data = torch.randn(1, 3, 224, 224).to('cuda')  
    inference_time = measure_inference_speed(model, input_data)

    data.append({
        "model_name": model_name,
        "number_of_parameters": num_params,
        "inference_speed (ms)": inference_time
    })

df = pd.DataFrame(data)
print(df)


/soft/AIDL/conda_envs/pytorch-200/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of ViTForImageClassification were not initialized from the model checkpoint at facebook/deit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([3, 192]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  model_name  number_of_parameters  inference_speed (ms)
0       DEiT               5524995              6.124608
1        ViT              85800963             22.955969


In [4]:
num_params_mobilenetv2 = calculate_parameters(mobile_net_v2_model)
num_params_bert = calculate_parameters(vit_model)

compression_ratio = num_params_bert / num_params_mobilenetv2
print(f"DEiT parameters: {num_params_mobilenetv2:,}")
print(f"BERT parameters: {num_params_bert:,}")
print(f"Compression ratio (ViT / DEiT): {compression_ratio:.2f}x") 

DEiT parameters: 5,524,995
BERT parameters: 85,800,963
Compression ratio (ViT / DEiT): 15.53x
